#**Goal Tracking Agent Powered by Flotorch:**

**Problem it solves & for whom:** Supports individuals who want to save or invest toward specific financial goals (e.g., buying a car, emergency fund, vacation) by tracking progress and adjusting monthly contributions and predicting the feasibility of achieveing the goal.

**Input:** Bank or credit card statements in PDF or CSV format, combined with user-defined goals and contributions stated in natural language.

**Output:** The output would be a downloadable  report in the form of a PDF that contains insights on the feasibility of achieving each goal, better approaches, and goal prioritization.

**Why it matters:** Financial goals often fail due to poor tracking and lack of motivation. This agent provides clear progress updates and data-driven plans, turning long-term goals into achievable, trackable milestones.

Install all required dependencies for the notebook. It includes data analysis and visualization tools (pandas, matplotlib, plotly), the flotorch framework for AI agent evaluation, and libraries for PDF generation and text extraction (fpdf2, pdfplumber).

In [1]:
# Pandas: data manipulation and analysis
# Matplotlib: static plotting library
# Plotly: interactive visualizations
!pip install pandas matplotlib plotly --quiet

# Flotorch (with "strands" extras): AI agent evaluation and tracing framework
# Version pinned to beta 2.1.0b1 for compatibility
!pip install flotorch[strands]==2.1.0b1

# fpdf2: library for generating PDFs programmatically
# pdfplumber: tool for extracting text and tables from PDFs
!pip install -q fpdf2 pdfplumber


INFO: pip is looking at multiple versions of opentelemetry-sdk to determine which version is compatible with other requirements. This could take a while.
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 101.1/101.1 kB 5.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 205.3/205.3 kB 10.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 139.4/139.4 kB 8.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 14.1/14.1 MB 90.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 208.0/208.0 kB 15.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 65.9/65.9 kB 4.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 132.3/132.3 kB 7.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 85.7/85.7 kB 3.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 88.0/88.0 kB 3.9 MB/s eta 0:00:00
  Attempting uninstall: wrapt
    Found existing installation: wrapt 2.0.0
    Uninstalling wrapt-2.0.0:
      Successfully uninsta

Imports essential modules: userdata from Google Colab for securely handling credentials, and FlotorchStrandsAgent from the Flotorch library to define and manage intelligent agents used in the workflow.

In [2]:
# Import utilities for securely accessing stored secrets or tokens in Google Colab
# (e.g., API keys or project credentials)
from google.colab import userdata

# Import the FlotorchStrandsAgent class from the flotorch.strands module
# This class enables creation and interaction with intelligent agent workflows
# that can be instrumented and evaluated via Flotorch
from flotorch.strands.agent import FlotorchStrandsAgent


This cell initializes the Goal Tracking Agent client inside the Flotorch framework.
It authenticates using the saved API key and prepares a connection to the goal-tracking-agent, which will later analyze savings goals, evaluate feasibility, and suggest actionable financial plans.

In [3]:
# Initialize a FlotorchStrandsAgent client for the "Goal Tracking Agent"
# This connects to the Flotorch Gateway using the stored API key and enables
# interaction with the deployed goal-tracking model.

goal_tracking_agent_client = FlotorchStrandsAgent(
    agent_name="goal-tracking-agent",            # Unique identifier for the agent
    api_key=userdata.get('flotorch_api_key'),    # Securely fetch the API key from Colab's userdata
    base_url="https://gateway.flotorch.cloud"    # Flotorch Gateway endpoint for API requests
)


2025-11-04 21:27:31 - flotorch.sdk.llm - INFO - FlotorchLLM initialized (model_id=flotorch/use-case-building:latest, base_url=https://gateway.flotorch.cloud)


2025-11-04 21:27:31 - flotorch.sdk.llm - INFO - FlotorchLLM initialized (model_id=flotorch/use-case-building:latest, base_url=https://gateway.flotorch.cloud)


2025-11-04 21:27:31 - flotorch.strands.llm - INFO - FlotorchStrandsModel initialized (model_id=flotorch/use-case-building:latest, base_url=https://gateway.flotorch.cloud)


2025-11-04 21:27:31 - flotorch.strands.llm - INFO - FlotorchStrandsModel initialized (model_id=flotorch/use-case-building:latest, base_url=https://gateway.flotorch.cloud)


This cell retrieves the deployed Goal Tracking Agent instance from the Flotorch Gateway.
Once initialized, the agent can be invoked to evaluate user-defined savings or investment goals, estimate feasibility, and provide monthly contribution plans or alerts based on spending behavior.

In [4]:
# Retrieve the active "Goal Tracking Agent" instance from the Flotorch Gateway.
# This returns a ready-to-use agent object that can process prompts
# related to financial goals, progress tracking, and feasibility analysis.

goal_tracking_agent = goal_tracking_agent_client.get_agent()


Handles file uploads and data loading.



*  If the user uploads a CSV, it is directly read into a pandas DataFrame.  
*  If PDFs are uploaded, up to 12 files are processed with pdfplumber to extract tabular data from each page.
All extracted tables are merged into one dataset, saved as merged_from_pdfs.csv, and loaded into the variable df for further processing.

In [5]:
# === FILE UPLOAD & LOAD (PDF or CSV) ==========================================
# This cell handles user uploads of either PDF or CSV files and loads them into a pandas DataFrame.
# If PDFs are provided, tables are extracted from up to 12 PDF files and merged into one CSV.
# The final DataFrame is stored as `df`.

import io
import os
import sys
import pandas as pd

# Check if running in Google Colab environment
try:
    from google.colab import files
except Exception as e:
    raise RuntimeError("This cell is designed for Google Colab. `google.colab` not available.") from e

# Ask user for input type (pdf or csv)
kind = input("Are you uploading pdf or csv? (pdf/csv): ").strip().lower()
if kind not in {"pdf", "csv"}:
    raise ValueError("Please type exactly 'pdf' or 'csv'.")

# =========================== CSV HANDLING ===========================
if kind == "csv":
    print("Upload a CSV file…")
    uploaded = files.upload()  # Prompt user to select file
    if not uploaded:
        raise RuntimeError("No file uploaded.")
    # Pick first uploaded file
    fname = next(iter(uploaded.keys()))
    if not fname.lower().endswith(".csv"):
        raise ValueError(f"Uploaded file '{fname}' is not a .csv")
    file_path = f"/content/{fname}"
    # Write to disk
    with open(file_path, "wb") as f:
        f.write(uploaded[fname])
    # Load into DataFrame
    df = pd.read_csv(file_path)
    print(f"Loaded CSV: {file_path} with {len(df):,} rows.")

# =========================== PDF HANDLING ===========================
else:
    print("Upload up to 12 PDF files…")
    uploaded = files.upload()
    if not uploaded:
        raise RuntimeError("No files uploaded.")
    pdf_files = [name for name in uploaded.keys() if name.lower().endswith(".pdf")]
    if not pdf_files:
        raise ValueError("No PDF files detected in the upload.")
    if len(pdf_files) > 12:
        raise ValueError(f"You uploaded {len(pdf_files)} PDFs. Please upload at most 12.")

    # Save each uploaded PDF to disk
    pdf_paths = []
    for name in pdf_files:
        path = f"/content/{name}"
        with open(path, "wb") as f:
            f.write(uploaded[name])
        pdf_paths.append(path)

    # Ensure pdfplumber is installed for table extraction
    try:
        import pdfplumber  # noqa
    except ImportError:
        print("Installing pdfplumber…")
        !pip -q install pdfplumber
        import pdfplumber  # noqa

    # Extract tables from each PDF page and collect into list
    all_tables = []
    import pdfplumber
    for p in pdf_paths:
        print(f"Extracting tables from {os.path.basename(p)} …")
        with pdfplumber.open(p) as pdf:
            for page_idx, page in enumerate(pdf.pages):
                try:
                    # Attempt structured extraction using line-based detection
                    tables = page.extract_tables(table_settings={
                        "vertical_strategy": "lines",
                        "horizontal_strategy": "lines",
                        "intersection_tolerance": 5,
                    })
                except Exception:
                    # Fallback to default extraction if above fails
                    tables = page.extract_tables()
                for t in tables or []:
                    if not t or len(t) < 1:
                        continue
                    # Identify headers and clean empty ones
                    header = t[0]
                    body = t[1:] if len(t) > 1 else []
                    header_clean = [
                        h if (h is not None and str(h).strip()) else f"col_{i}"
                        for i, h in enumerate(header)
                    ]
                    df_tbl = pd.DataFrame(body, columns=header_clean)
                    # Keep table if it has meaningful data
                    if df_tbl.shape[1] >= 2 and df_tbl.shape[0] >= 1:
                        all_tables.append(df_tbl)

    # Check extracted results
    if not all_tables:
        raise RuntimeError("No tables detected in the uploaded PDFs. Please ensure your PDFs contain tabular data.")

    # Merge all extracted tables into one DataFrame
    merged_df = pd.concat(all_tables, ignore_index=True)

    # Save to CSV for reproducibility
    file_path = "/content/merged_from_pdfs.csv"
    merged_df.to_csv(file_path, index=False)

    # Assign merged data to `df`
    df = merged_df
    print(f"Merged {len(pdf_paths)} PDFs → {file_path} with {len(df):,} rows.")


2025-11-04 21:27:32 - numexpr.utils - INFO - NumExpr defaulting to 2 threads.


2025-11-04 21:27:32 - numexpr.utils - INFO - NumExpr defaulting to 2 threads.
Are you uploading pdf or csv? (pdf/csv): csv
Upload a CSV file…


Saving personal_transactions.csv to personal_transactions.csv
Loaded CSV: /content/personal_transactions.csv with 806 rows.


Cleans and standardizes the dataset. It ensures that every file has consistent column names for Date, Amount, and Description, even if the uploaded data used different labels. After normalizing, it previews the first 10 rows of the cleaned DataFrame to confirm successful parsing.

In [6]:
# === NORMALIZE COLUMNS & PREVIEW =============================================
# Soft-parse a date column, normalize amount column name, and ensure a description column exists.
# Expects `df` and `file_path` to already be defined by the previous upload cell.

import pandas as pd
from IPython.display import display

# --- Identify and parse date columns ---
# Look for a column that matches common date naming patterns
date_cols = [c for c in df.columns if str(c).lower() in ["date", "posted_date", "transaction_date", "datetime"]]
if date_cols:
    # Convert recognized date column to datetime format (coercing invalid entries to NaT)
    df[date_cols[0]] = pd.to_datetime(df[date_cols[0]], errors="coerce")
    # Rename first matching column to standard "Date"
    df.rename(columns={date_cols[0]: "Date"}, inplace=True)
else:
    # If no date column exists, create one filled with NaT to maintain structure
    df["Date"] = pd.NaT  # absent

# --- Normalize amount column ---
amt_col = None
# Search for a column name representing transaction amount
for c in df.columns:
    if str(c).lower() in ["amount", "amt", "value", "transaction_amount"]:
        amt_col = c
        break
# Raise an error if no recognizable amount column is found
if amt_col is None:
    raise ValueError("Could not find an amount-like column (e.g., Amount). "
                     "Please rename your amount column to 'Amount' and re-run.")

# Standardize amount column name to "Amount"
df.rename(columns={amt_col: "Amount"}, inplace=True)

# --- Ensure a description/merchant column exists ---
desc_col = None
# Try finding columns that likely contain text descriptions or merchant names
for c in df.columns:
    if str(c).lower() in ["description", "merchant", "payee", "narration", "memo"]:
        desc_col = c
        break
if desc_col is None:
    # If none exist, create a fallback "Description" column with row numbers
    df["Description"] = "Row-" + (df.reset_index().index + 1).astype(str)
    desc_col = "Description"

# --- Display summary and preview ---
print("Loaded/Prepared data from:", file_path)
print("Columns normalized to include: ['Date', 'Amount', 'Description', ...]")
display(df.head(10))  # Show first 10 rows for verification


Loaded/Prepared data from: /content/personal_transactions.csv
Columns normalized to include: ['Date', 'Amount', 'Description', ...]


,Date,Description,Amount,Transaction Type,Category,Account Name
0,2018-01-01,Amazon,11.11,debit,Shopping,Platinum Card
1,2018-01-02,Mortgage Payment,1247.44,debit,Mortgage & Rent,Checking
2,2018-01-02,Thai Restaurant,24.22,debit,Restaurants,Silver Card
3,2018-01-03,Credit Card Payment,2298.09,credit,Credit Card Payment,Platinum Card
4,2018-01-04,Netflix,11.76,debit,Movies & DVDs,Platinum Card
5,2018-01-05,American Tavern,25.85,debit,Restaurants,Silver Card
6,2018-01-06,Hardware Store,18.45,debit,Home Improvement,Silver Card
7,2018-01-08,Gas Company,45.00,debit,Utilities,Checking
8,2018-01-08,Hardware Store,15.38,debit,Home Improvement,Silver Card
9,2018-01-09,Spotify,10.69,debit,Music,Platinum Card


Prompts the user to enter personal financial goals (e.g., saving targets, debt payoffs, or investments).
The input text provides the Goal Tracking Agent with context for analysis — including the goal type, target amount, and planned monthly contributions — to generate feasibility and progress recommendations later in the workflow.

In [ ]:
# === GOALS INPUT CELL =========================================================
# This cell collects user-defined financial goals as plain text input.
# Each goal should describe a target amount and planned monthly contribution.

# Display sample input format to guide the user
print("""
Best format of sample input for efficient goal mapping:

Save $5k for an emergency fund by putting $300 per month.
Pay off credit card balance of $2,000 with $600/mo.
Vacation in Japan: target 2500; plan to set aside 200 monthly.
Down payment for a car: goal $10k; contributing $400 pm.
""")

# Prompt the user to enter their goals (multi-line text input)
user_goal_text = input("Enter your goals here (press Enter when done):\n")



Best format of sample input for efficient goal mapping:

Save $5k for an emergency fund by putting $300 per month.
Pay off credit card balance of $2,000 with $600/mo.
Vacation in Japan: target 2500; plan to set aside 200 monthly.
Down payment for a car: goal $10k; contributing $400 pm.



This cell converts the user’s free-form goal descriptions into structured dictionaries containing:

*   name – cleaned goal title (e.g., “Emergency Fund”)
*   target – total goal amount
*   monthly_contribution – monthly saving or payment value

It uses regular expressions, context keywords, and heuristic text cleaning to identify patterns like “Save $5000 for an emergency fund by saving $300/mo”.
The parsed results are stored in the variable goals, ready for analysis by the Goal Tracking Agent.

In [8]:
# === GOALS PARSER CELL =======================================
# This cell parses natural-language goal descriptions into structured data.
# It extracts goal names, target amounts, and monthly contributions
# from free-form text like "Save $5k for an emergency fund with $300 per month."

import re
from typing import List, Dict, Optional

# Regular expression to detect monetary values (supports $, commas, decimals, and 'k' suffix)
_AMOUNT_RE = re.compile(r"(?P<prefix>\$)?(?P<number>(?:\d{1,3}(?:,\d{3})+|\d+(?:\.\d+)?)(?:[kK])?)")

# Common context keywords to identify monthly contributions or goal targets
_MONTHLY_HINTS = ["per month", "monthly", "/mo", "pm", "each month", "a month", "mo"]
_TARGET_HINTS  = ["target", "goal"]

# Noise words and phrases often found before or around goal names
_LEADING_NOISE = [
    "save","set aside","build","grow","create","fund","target","goal","for","towards","to",
    "pay","pay off","payoff","clear","reduce","plan to","contributing","contribution","with"
]

# Common goal-related phrases and aliases for normalization
_COMMON_NAMES = [
    "emergency fund","credit card","credit-card","loan","vacation","travel",
    "down payment","downpayment","car","house","wedding","education","medical"
]
_ALIAS = {"credit card": "Credit Card Payoff", "down payment": "Down Payment"}

# --- Utility functions ---

def _normalize_amount(token:str)->float:
    """Normalize numeric strings like '$5k', '2,000', '300.50' into floats."""
    token = token.strip().replace(",", "")
    if token.startswith("$"):
        token = token[1:]
    if token.lower().endswith("k"):
        return float(token[:-1]) * 1000
    return float(token or 0)

def _find_amounts(text:str):
    """Return all detected monetary values in a text."""
    return list(_AMOUNT_RE.finditer(text))

def _is_monthly_context(s:str)->bool:
    """Detect whether the nearby text suggests a monthly payment context."""
    return any(h in s.lower() for h in _MONTHLY_HINTS)

def _is_target_context(s:str)->bool:
    """Detect whether the nearby text suggests a total goal or target context."""
    return any(h in s.lower() for h in _TARGET_HINTS)

def _smart_split(text:str)->List[str]:
    """Split long paragraphs into manageable sentence-like parts."""
    lines = [ln.strip() for ln in text.splitlines() if ln.strip()]
    if len(lines) == 1 and len(lines[0]) > 200:
        parts = [p.strip() for p in re.split(r"[.!?—–-]+\s+", lines[0]) if p.strip()]
        return parts
    return lines

def _clean_name(raw:str)->str:
    """Remove numeric and noise terms to extract a clean, readable goal name."""
    s = re.sub(_AMOUNT_RE, " ", raw)
    s = re.sub(r"[:;,./()\-\—\–]+", " ", s).lower().strip()
    for phrase in _COMMON_NAMES:
        if phrase in s:
            s = phrase
            break
    for phrase in sorted(_LEADING_NOISE, key=len, reverse=True):
        s = re.sub(rf"^\b{re.escape(phrase)}\b\s*", "", s)
    if " for " in s:
        s = s.split(" for ", 1)[-1]
    s = re.sub(r"\s{2,}", " ", s).strip() or "General Savings"
    for k, v in _ALIAS.items():
        if k in s:
            s = v
            break
    return " ".join(w.capitalize() for w in s.split())

# --- Core Parser ---
def parse_natural_language_goals(text:str)->List[Dict[str,Optional[float]]]:
    """Parse goal descriptions into structured dictionaries with name, target, and monthly_contribution."""
    results = []
    for line in _smart_split(text):
        amounts = _find_amounts(line)
        if not amounts:
            continue
        spans = [(m.start(), m.end(), _normalize_amount(m.group(0))) for m in amounts]
        monthly, target = None, None

        # Analyze context around each detected number
        for (s, e, v) in spans:
            window = line[max(0, s - 24):min(len(line), e + 24)]
            if monthly is None and _is_monthly_context(window):
                monthly = v
            if target is None and _is_target_context(window):
                target = v

        # Guess target if not explicitly found
        if target is None:
            candidates = [v for (_, _, v) in spans if v != monthly]
            if candidates:
                target = max(candidates)

        # Handle cases with two numeric values but no clear labels
        if monthly is None and target is None and len(spans) == 2:
            a, b = sorted([v for (_, _, v) in spans])
            monthly, target = a, b

        # Handle single numeric + monthly keyword case
        if monthly is None and len(spans) == 1 and _is_monthly_context(line):
            monthly = spans[0][2]

        # Clean goal name for consistency
        name = _clean_name(line)

        # Store results only if at least one valid value found
        if target is not None or monthly is not None:
            results.append({"name": name, "target": target, "monthly_contribution": monthly})

    # Merge duplicate goals (same name) intelligently
    merged = {}
    for r in results:
        k = r["name"]
        if k not in merged:
            merged[k] = r
        else:
            if (merged[k]["target"] or 0) < (r["target"] or 0):
                merged[k]["target"] = r["target"]
            if merged[k]["monthly_contribution"] is None and r["monthly_contribution"] is not None:
                merged[k]["monthly_contribution"] = r["monthly_contribution"]

    return list(merged.values())

# --- Driver ---
# Run parsing only if the user provided goal text
if not user_goal_text.strip():
    print("No input detected. Please run the first cell and enter goals.")
else:
    goals = parse_natural_language_goals(user_goal_text)
    from pprint import pprint
    print("\nParsed goals (assignable as `goals`):")
    pprint(goals)
    globals()["goals"] = goals



Parsed goals (assignable as `goals`):
[{'monthly_contribution': 300.0, 'name': 'Emergency Fund', 'target': 2500.0}]


Computes an approximate average monthly spend from the last 3 months of transactions (if dates exist) and embeds it, along with the parsed goals, into a prompt for the Goal Tracking Agent. The prompt asks the agent to judge feasibility, infer income, suggest monthly plans, estimate ETAs, and surface goal-related alerts, returning concise bullet-point feedback for each goal.

In [9]:
# Provide minimal spending context for feasibility
avg_monthly_spend = None
if pd.notna(df["Date"]).any():
    # Work only with rows that have a valid Date
    tmp = df.dropna(subset=["Date"]).copy()
    # Convert Date to a monthly period (e.g., 2025-01, 2025-02)
    tmp["Month"] = tmp["Date"].dt.to_period("M")
    # Aggregate total Amount by month
    monthly_spend = tmp.groupby("Month")["Amount"].sum()
    # assume outflows are negative or positive depending on your CSV; keep magnitude
    # Use the last 3 months of spend to approximate an average monthly spend
    avg_monthly_spend = float(monthly_spend.tail(3).mean()) if len(monthly_spend) else None

# Build the prompt sent to the Goal Tracking Agent
goal_prompt = f"""
You are the Goal Tracking Agent.

User goals (simple list):
{goals}

Avg monthly spend (approx): {avg_monthly_spend}

For each goal:
- Judge feasibility in plain text.
- Define montly income and then give a detailed breakdown of how the goal can be met.
- Suggest a monthly plan (use given monthly_contribution; adjust if needed).
- Estimate ETA to reach the target (simple math is fine).
- Add any goal_alerts (like overspending risks, too-small contributions, etc.).

Keep the output as short bullets per goal. No JSON.
"""


Configure logging behavior to keep notebook output clean. It removes any existing log handlers from Flotorch and Strands modules, disables log propagation, and reduces verbosity to the WARNING level — ensuring that only essential messages appear during execution.

In [10]:
import logging

# Remove any existing Flotorch or Strands log handlers
# This prevents duplicate log messages or excessive verbosity in Colab output
for name in ["flotorch", "strands"]:
    logger = logging.getLogger(name)
    logger.handlers.clear()   # Clear existing handlers
    logger.propagate = False  # Prevent propagation to the root logger

# Optionally reduce log verbosity for cleaner output
# Setting both Flotorch and Strands modules to WARNING hides debug/info logs
logging.getLogger("flotorch").setLevel(logging.WARNING)
logging.getLogger("strands").setLevel(logging.WARNING)


Invokes the Goal Tracking Agent with the constructed goal_prompt.
The agent uses the user’s defined goals and recent spending data to produce a detailed, human-readable breakdown — evaluating feasibility, monthly plans, timelines, and alerts for each goal. The result is stored in the variable response for further display or PDF report generation.

In [11]:
# Send the prepared goal_prompt to the Goal Tracking Agent
# The agent will analyze each user goal based on:
# - Target amount and monthly contribution
# - Estimated monthly spending context
# - Feasibility and time-to-achieve (ETA)
# - Any financial risks or imbalances detected

response = goal_tracking_agent(goal_prompt)


**Emergency Fund Goal Analysis:**

• **Feasibility**: HIGHLY FEASIBLE - $2,500 target is reasonable for emergency fund (should be 3-6 months expenses, but this covers ~2.6 weeks of current spending)

• **Monthly Income Needed**: Based on $11,299 monthly spend, you need ~$11,600+ monthly income to sustain current lifestyle + $300 goal contribution

• **Monthly Plan**: 
  - Keep $300/month contribution as planned
  - This represents 2.7% of monthly spending - very manageable
  - Set up automatic transfer on payday

• **ETA**: 8.3 months ($2,500 ÷ $300 = 8.33 months)

• **Goal Alerts**:
  - ⚠️ **CRITICAL**: $2,500 emergency fund is severely inadequate - should be $34,000-$68,000 (3-6 months of expenses)
  - Consider this a "starter emergency fund" and plan Phase 2 goal for full 3-month coverage
  - High monthly spending ($11,299) creates significant financial risk if income disrupted
  - Recommend reviewing spending categories to potentially increase emergency fund contributions

Generates a polished Goal Tracking PDF Report from the agent’s analysis.
It cleans the response text (removing markdown, emojis, and formatting artifacts) and then uses fpdf2 to produce a structured, readable “Goal Tracking Analysis Report.”
The final PDF includes sections like Monthly Income Analysis, Goal Feasibility, and Alerts, displayed with proper formatting and made available as a downloadable link directly in Colab.

In [12]:
from fpdf import FPDF
from fpdf.enums import XPos, YPos
from IPython.display import HTML, display
from io import BytesIO
import base64, re

# Extract raw text from the agent response
# The 'response' variable comes from: response = goal_tracking_agent(goal_prompt)
raw = response

# Handle different response structures (Strands, Flotorch, or plain string)
if hasattr(raw, "message"):
    raw = raw.message
if isinstance(raw, dict) and "content" in raw:
    try:
        # Safely extract the 'text' field from nested content
        raw = raw["content"][0].get("text", str(raw))
    except Exception:
        raw = str(raw)

# Ensure we’re working with a string version of the content
text = str(raw)

# Clean markdown + emojis that FPDF can't handle well
# Replace symbols and emojis with safe ASCII equivalents
text = text.replace("•", "-")
text = text.replace("–", "-").replace("—", "-")
text = text.replace("⚠️", "[ALERT]").replace("⚠", "[ALERT]")
text = text.replace("🚨", "[CRITICAL]")

# Remove markdown formatting characters (e.g., *, _, `, #)
clean_text = re.sub(r"[`*_#]+", "", text)
lines = [ln.strip() for ln in clean_text.splitlines() if ln.strip()]

# Build the PDF
pdf = FPDF()
pdf.add_page()
pdf.set_auto_page_break(auto=True, margin=15)

# Add report title
pdf.set_font("Helvetica", style="B", size=16)
pdf.cell(0, 10, "Goal Tracking Analysis Report",
         new_x=XPos.LMARGIN, new_y=YPos.NEXT, align="C")
pdf.ln(6)

# Add short introduction
pdf.set_font("Helvetica", size=12)
pdf.multi_cell(0, 8, "Generated by Goal Tracking Agent based on your goals and estimated monthly income:")
pdf.ln(4)

# Format and write the main report content
for line in lines:
    # Identify section headers or goal names and make them bold
    if (
        line.endswith(":")
        or line.lower().startswith("monthly income analysis")
        or line.lower().startswith("goal analysis")
        or line.lower().startswith("recommended reallocation")
        or re.match(r"^[A-Z].*\(\$", line)  # e.g. "Emergency Fund ($5,000)"
    ):
        pdf.set_font("Helvetica", style="B", size=12)
        pdf.multi_cell(0, 8, line)
        pdf.ln(2)
    # Format bullet points with indentation
    elif line.startswith("-"):
        pdf.set_font("Helvetica", size=11)
        pdf.multi_cell(0, 7, "   " + line.lstrip("- ").strip())
    # Normal body text
    else:
        pdf.set_font("Helvetica", size=11)
        pdf.multi_cell(0, 7, line)
    pdf.ln(1)

# Export the PDF to memory and display a download link
buf = BytesIO()
pdf.output(buf)
pdf_bytes = buf.getvalue()

# Convert to base64 for in-browser download
b64 = base64.b64encode(pdf_bytes).decode("ascii")
href = f'<a download="Goal_Tracking_Report.pdf" href="data:application/pdf;base64,{b64}" target="_blank">Download Goal Tracking Report (PDF)</a>'
display(HTML(href))
